In [14]:
#Import necessary libraries
from flask import Flask, render_template, Response
import cv2
#Initialize the Flask app
app = Flask(__name__)

camera = cv2.VideoCapture(0)
'''
for ip camera use - rtsp://username:password@ip_address:554/user=username_password='password'_channel=channel_number_stream=0.sdp' 
for local webcam use cv2.VideoCapture(0)
'''
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import cv2
from keras.models import load_model

facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
font=cv2.FONT_HERSHEY_COMPLEX

threshold=0.90
model = load_model('ModelWeight.h5')

def preprocessing(img):
    img=img.astype("uint8")
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.equalizeHist(img)
    img = img/255
    return img


def get_className(classNo):
    if classNo==0:
        return "Helmet"
    elif classNo==1:
        return "No Helmet"
    
def gen_frames():  
    while True:
        success, frame = camera.read()  # read the camera frame
        
        if not success:
            break
        else:
            try:
                faces = facedetect.detectMultiScale(frame,1.3,5)

                for x,y,w,h in faces:
                    crop_img=frame[y:y+h,x:x+h]
                    img=cv2.resize(crop_img, (32,32))
                    img=preprocessing(img)
                    img=img.reshape(1, 32, 32, 1)

                    prediction=model.predict(img)
                    classIndex=model.predict_classes(img)
                    probabilityValue=np.amax(prediction)
                    if probabilityValue>threshold:
                        if classIndex==0:
                            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                            cv2.rectangle(frame, (x,y-40),(x+w, y), (0,255,0),-2)
                            cv2.putText(frame, str(get_className(classIndex)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)
                        elif classIndex==1:
                            cv2.rectangle(frame,(x,y),(x+w,y+h),(50,50,255),2)
                            cv2.rectangle(frame, (x,y-40),(x+w, y), (50,50,255),-2)
                            cv2.putText(frame, str(get_className(classIndex)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)

                    ret, buffer = cv2.imencode('.jpg', frame)
                    frame = buffer.tobytes()
                    yield (b'--frame\r\n'
                           b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result
                
            except:
                continue
        k=cv2.waitKey(1)
        if k==ord('q'):
            break

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template("index.html")


app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Nov/2021 18:05:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2021 18:05:48] "GET /video_feed HTTP/1.1" 200 -
Exception ignored in: <generator object gen_frames at 0x000001BFE1E05200>
Traceback (most recent call last):
  File "C:\Users\Asus ROG\anaconda3\lib\site-packages\werkzeug\serving.py", line 320, in execute
    application_iter.close()
RuntimeError: generator ignored GeneratorExit
